In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import nbformat as nbf
import json
import os



In [3]:
class Payload:
    def __init__(self):
        self.data = None
    
    def set_payload(self, data):
        self.data = data

    def get_payload(self):
        return self.data
    

In [17]:
def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.page_load_strategy = 'none'
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def get_info(url, class_name_presence):
    data = []
    driver = setup_driver()
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, class_name_presence)))
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        if class_name_presence == 'contents':
            data = getting_list_data(soup)

        elif class_name_presence == 'QuestionDetails__contents':
            getting_data_info(url, soup)

    except TimeoutException:
        print("Loading took too much time!")

    finally:
         driver.quit()

    return data

def getting_list_data(soup):
    data = []
    for list in soup.find_all('a', 'group contents'):
        info = {}
        details = list.find_all('div')
        info['link'] = 'https://platform.stratascratch.com' + list.get('href')
        info['title'] = details[4].getText().replace('/','-').replace('?', '')
        info['company'] = details[1].getText()
        info['difficulty'] = details[2].getText()
        info['question_type'] = details[3].getText()
        data.append(info)

    return data

def getting_data_info(url, soup):
    payload = []
    title = soup.find('h1','QuestionMetadata__h1').getText().replace('/','-').replace('?', '')
    question_des_list = soup.find_all('div', 'QuestionMetadata__metadata')
    question_content_list = [child.get_text(strip=True) for child in question_des_list[0].children if child.get_text(strip=True)]

    payload.append(nbf.v4.new_markdown_cell(f"### **Link:** {url}"))
    payload.append(nbf.v4.new_markdown_cell(f"### **ID:** {question_content_list[-2]}".replace("ID: ", '')))
    payload.append(nbf.v4.new_markdown_cell(f"# {title}"))
    payload.append(nbf.v4.new_markdown_cell(f"### **Difficulty:** {question_content_list[-4]}"))
    payload.append(nbf.v4.new_markdown_cell(f"### **Question Type:** {question_content_list[-3]}"))
    payload.append(nbf.v4.new_markdown_cell(f"### **Company:** {', '.join(question_content_list[0:len(question_content_list) - 4])}"))
    payload.append(nbf.v4.new_markdown_cell(f"## **Description:**"))
    payload.append(nbf.v4.new_markdown_cell(soup.find('div','QuestionMetadata__question').find('p').getText()))
    payload.append(nbf.v4.new_markdown_cell(f"## **Solution:**"))
    payload.append(nbf.v4.new_code_cell())

    
    base_folder = r'Non Coding Notebook'
    sub_folder = f'{question_content_list[-3]}\\' + question_content_list[-4]
    create_notebook(payload, f'{title}.ipynb', os.path.join(base_folder, sub_folder))

def init_process():
    list_question = []

    for i in range(1, 5):
        url = f"https://platform.stratascratch.com/technical?page_size=100&order_field=difficulty&page={i}"
        list_question.extend(get_info(url, 'contents'))

    create_folder('Non Coding Notebook')
    json_dump('non_coding_question_list.json', list_question)

def json_dump(file, data):
    with open(file, 'w+') as f:
        json.dump(data, f)
    
def create_folder(folder_name, folder_path=""):
    new_folder = os.path.join(folder_path, folder_name)
    if not os.path.isabs(new_folder):
        current_path = os.getcwd()
        new_folder = os.path.join(current_path, new_folder)

    if not os.path.exists(new_folder):
        os.makedirs(new_folder)

def create_notebook(payload, name, path):
    notebook = nbf.v4.new_notebook()
    notebook['cells'] = payload

    with open(path + '/' + name, 'w', encoding="utf-8") as f:
        nbf.write(notebook, f)


In [6]:
init_process()

In [21]:
base_folder = r'Non Coding Notebook'

file_path = os.path.join(os.getcwd(), 'logs_none_coding')
file_list = [f for f in os.listdir(file_path) if os.path.isfile(os.path.join(file_path, f))]

with open('non_coding_question_list.json', 'r+') as list_non_coding:
    for la in json.load(list_non_coding):
        if la['title'] not in file_list:
            create_folder(la['question_type'], base_folder)
            create_folder(la['difficulty'],  os.path.join(base_folder, la['question_type']))
            get_info(la['link'],'QuestionDetails__contents')
            json_dump('logs_none_coding/' + la['title'], '')
            

FileNotFoundError: [Errno 2] No such file or directory: 'Non Coding Notebook\\Statistics\\Medium/A/B Testing.ipynb'